## Code Binding summaries using Jinja2 and Python Modele

- Fetch SD file from IG
- Transform to Python model
- use Jinja2 template to create a summary markdown file
- save markdown file



### import python modules including R4 fhirclient models

In [ ]:
from fhirclient.r4models import structuredefinition as SD
from fhirclient.r4models import narrative as N
from fhirclient.r4models import valueset as VS
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from pprint import pprint
from stringcase import snakecase, titlecase
from collections import namedtuple
from pandas import *
from datetime import date
from jinja2 import Environment, FileSystemLoader, select_autoescape
from commonmark import commonmark
from IPython.display import display, HTML, Markdown
from lxml import etree

### Get US Core SD file names

In [ ]:
canon = 'http://hl7.org/fhir/us/core/'  # ig canonical base

#in_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/output/StructureDefinition-'
#f_name = 'us-core-inspired-oxygen'
ig_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core.r4#dev/package'
ig_path = "//ERICS-AIR-2/ehaas/.fhir/packages/hl7.fhir.us.core.r4#dev/package"

ig_files = [x for x in os.listdir(ig_path) if x.startswith("StructureDefinition")]
ig_files

### Get matchin FHIR core SD file names

In [ ]:


#core_path = '/Users/ehaas/.fhir/packages/hl7.fhir.core#current/package'
core_path = '/Users/ehaas/.fhir/packages/hl7.fhir.core#4.0.0/package'
core_files = [x for x in os.listdir(core_path) if x.startswith("StructureDefinition")]

core_files

In [ ]:
def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}') as f:
        r = f.read()
        return(loads(r))
   

In [ ]:
 
def expand(vs):
    print(vs)
    #fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    #fhir_test_server = 'http://test.fhir.org/r4'
    fhir_test_server = 'https://r4.ontoserver.csiro.au/fhir'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
          url = vs.split('|')[0],
          count = 20,
            )
    
    r = get(f'{fhir_test_server}/ValueSet/$expand', params = params, headers = headers )
    # return r.status_code
    # view  output

    
    if r.status_code == 200:
        return VS.ValueSet(r.json(),strict= False)

In [ ]:
j_list = []    
for i in ig_files:
    sd_dict = open_file(ig_path,i)
    try:
        core_sd_dict = open_file(core_path,i.replace("-us-core",""))
    except FileNotFoundError:
        pass
    try:
        sd = SD.StructureDefinition(sd_dict)
        diff_e = [i.path for i in sd.differential.element]
        for j in sd.snapshot.element:
            if j.min == 1 and j.binding and j.path in diff_e:
                j_url = f'{canon}{i.replace(".json","-definitions.html#")}{j.path}'
                if j.binding.strength == 'required' and j.min ==1:
                    vs = expand(j.binding.valueSet)
                    try:
                        core_sd = SD.StructureDefinition(core_sd_dict)
                        j_core = next(k for k in core_sd.snapshot.element if k.path == j.path)
                        j_core_url = f'https://build.fhir.org/{j.path.split(".")[0].lower()}-definitions.html#{j.path}'
                    except StopIteration:
                        j_core = None
                    j_list.append((j,vs,j_url,j_core, j_core_url))
    except FHIRValidationError as e:
        print(e)
    except AttributeError:
        pass 
j_list

In [ ]:
for i in j_list:
    pprint(i[1].__dict__)

### Using Jinja2 Template create md file for summary view

In [ ]:
k_list = []
for j in j_list:
    try:
     k_list.append(tuple(list(j) + [c.code for c in j[1].expansion.contains if c.code in ['unknown','other']]))
    except AttributeError:
        k_list.append(j)
k_list

In [ ]:
in_path = ''
template_path = 'binding_summary_template4.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown

template = env.get_template(template_path)



d = template.render(elements = k_list)
display(Markdown(d))

### write to file

In [ ]:
out_path = ''
out_file = 'binding_summary5.md'

def write_file(path, f_name, data): # write file
    with open(f'{path}{f_name}', 'w') as f:
        f.write(data)

write_file(out_path,out_file,d)

## Fin